In [10]:
# -*- coding: utf-8 -*-
"""
Created on Tue Jun  4 17:38:46 2019

@author: dkang
"""
import torch.backends.cudnn as cudnn
import torch.nn.init as init
import torch.utils.data as data
import numpy as np
import argparse
import torchvision
import visdom
import pickle

from data import *
from utils.augmentations import SSDAugmentation
from layers.modules import MultiBoxLoss

from ssd import build_ssd
from box_ssd2 import build_ssd_box2
from ssd_full_box import build_ssd_full_box

import os
import sys
import time
import torch
from torch.utils.data import SubsetRandomSampler
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
from utils import metrics

%reload_ext autoreload
%autoreload 2

def str2bool(v):
    return v.lower() in ("yes", "true", "t", "1")


cfg = voc

dataset_train = VOCDetection(root='/home/harryemeric/Project/Code/data/VOCdevkit',
                               image_sets = [('2012', 'trainval')],
                               transform=SSDAugmentation(cfg['min_dim'],
                                                         MEANS))


data_loader_train = data.DataLoader(dataset_train, 2,
                                  num_workers=0,
                                  shuffle=False, 
                                  collate_fn=detection_collate,
                                  pin_memory=True)

ssd_net = build_ssd_box2('train', cfg['min_dim'], cfg['num_classes'])
net = ssd_net

In [4]:
net(next(iter(data_loader_train))[0])

(tensor([[[ 0.3386,  0.1866, -0.3954, -0.0122],
          [ 0.1867,  0.2186,  0.1364, -0.5054],
          [-0.5034,  0.4894, -0.2949,  0.1638],
          ...,
          [-0.0108, -0.0145,  0.0157,  0.0081],
          [ 0.0014, -0.0196,  0.0019,  0.0067],
          [-0.0209,  0.0123, -0.0041,  0.0030]],
 
         [[-0.0141, -0.2737, -0.6129, -0.4052],
          [ 0.2075,  0.0900,  0.2979, -0.2289],
          [-0.8989,  0.2127, -0.0412,  0.0848],
          ...,
          [-0.0108, -0.0145,  0.0157,  0.0081],
          [ 0.0014, -0.0196,  0.0019,  0.0067],
          [-0.0209,  0.0123, -0.0041,  0.0030]]], grad_fn=<ViewBackward>),
 tensor([[[-0.0664, -0.4813,  0.0419,  ...,  0.5142,  0.4676,  0.3101],
          [-0.5871,  0.4704, -0.2780,  ..., -0.2277,  0.2635, -0.1552],
          [ 0.1667,  0.4406, -0.5340,  ...,  0.3721, -0.3927, -0.5965],
          ...,
          [ 0.0176,  0.0026, -0.0062,  ...,  0.0212, -0.0156,  0.0137],
          [ 0.0197, -0.0176, -0.0118,  ...,  0.0118,  0.0205,

In [11]:
ssd_full_box = build_ssd_full_box('train', size=300, num_classes=21)

In [18]:
out = ssd_full_box(next(iter(data_loader_train))[0])

In [20]:
len(out[0][0])

8732

In [21]:
pwd

'/home/harryemeric/Project/cs231n_project_box_convolution/ssd.pytorch'

In [24]:
torch.save(ssd_full_box.vgg.state_dict(),'/home/harryemeric/Project/Code/weights/FullSSDZeros.pth')